In [32]:
#importing libraries that are neccessary
from bs4 import BeautifulSoup as bs
import requests,openpyxl

In [33]:
#creating the excel sheet
excel = openpyxl.Workbook()
print(excel.sheetnames)
sheet = excel.active
#changing the title of the sheet
sheet.title = 'Top Grossing Movies'
print(excel.sheetnames)
#creating the columns for our sheet
sheet.append(['Movie Rank','Movie Name','Year Released','Gross Amount','Rating from Top 250'])

['Sheet']
['Top Grossing Movies']


In [34]:
#all the different pages that have the information we want
links = ['https://www.imdb.com/search/title/?groups=top_250&sort=boxoffice_gross_us,desc',
         'https://www.imdb.com/search/title/?groups=top_250&sort=boxoffice_gross_us,desc&start=51&ref_=adv_nxt',
         'https://www.imdb.com/search/title/?groups=top_250&sort=boxoffice_gross_us,desc&start=101&ref_=adv_nxt',
         'https://www.imdb.com/search/title/?groups=top_250&sort=boxoffice_gross_us,desc&start=151&ref_=adv_nxt',
         'https://www.imdb.com/search/title/?groups=top_250&sort=boxoffice_gross_us,desc&start=201&ref_=adv_nxt']

#making an array for the results to go into
ordered_list = []
#for loop that will grab each link and extract the information from those pages
for link in links:
    url = link

    html = requests.get(url)

    #making url into beautiful soup object
    s = bs(html.content,'html.parser')

    #using prettify to make html code readable
    contents = s.prettify()

    #pointing to specific box where all the information is stored in the html code
    movie_list = s.find(class_='lister-list')

    #making python find all the boxes where the movie information lies in the html code 
    movie_detail = movie_list.find_all('div',class_='lister-item-content')

    #for loop to extract all the information i want for the top 250 movies
    for movie in movie_detail:
        #rank variable will return the rank of the movie
        rank = movie.find('span',class_='lister-item-index unbold text-primary').text
        #name variable will return the name of the movie
        name = movie.find('a').text
        #year variable will return the year the movie was released
        year = movie.find('span',class_='lister-item-year text-muted unbold').text.strip("(I) ")
        #desired_gross will return the amount that movie made, have to use various functions to clean the text
        #making a variable to check if the money sign '$' is in extra_info, then extract the gross amount
        money_sign = '$'
        extra_info = movie.find('p',class_='sort-num_votes-visible').text
        if money_sign in extra_info:
            desired_gross = extra_info.split('$')[1] #split will grab the text after the $ which will be the gross amount
            desired_gross = desired_gross.partition('|')[0] #partition will grab everything after the | character
            desired_gross = desired_gross.strip('\n') #stripping away the '\n' that pops up sometimes after the results
        else:
            desired_gross= "NULL" #if no dollar sign is found then the amount is NULL, the website doesnt have it
        # rating variable will grab and store their rank in the top 250 movie ratings
        rating = extra_info.split('Top 250:')[1]
        rating = rating.strip('\n')
        #putting all the information im grabbing into the array i made earlier, and in a specific order to make it look nice 
        ordered_list.append([rank,name,year,desired_gross,rating])
        #also storing results onto the excel sheet made above
        sheet.append([rank,name,year,desired_gross,rating])
#printing ordered_list to make sure results are correct
ordered_list
#saving the excel sheet
excel.save('IMDB Top Grossing Movies.xlsx')